# Simulating noisy circuits

In this notebook we present how run noisy simulations by defining noise models and adding then to a ``QCircuit``.

If you are not familiar with the basic concepts of noise for quantum systems, we recommand you this introduction `TODO`

In [94]:
from mpqp import QCircuit
from mpqp.gates import *
from mpqp.noise import Depolarizing
from mpqp.execution import *

### Instantiate your noise model

In MPQP, the abstract class ``NoiseModel`` represents noisy channels acting on the qubits of the circuit, either after each gate application, or as an interaction with the environement (what is called idle noise). Each predefined noise model should extend this class, which has common attributes ``targets`` (indicating the indices of the qubits affected by this noise model) and the optional ``gates`` (indicating specific gates after which the noise will be applied)

For example, if one wants to apply a depolarizing noise on the circuit, he can use the class ``Depolarizing``. We must to specify as first argument the probability, or the error rate of the channel, and as second argument the qubits that will be targetted by this instance.

In [ ]:
Depolarizing(0.5, [0, 1, 2])

``dimension`` is by default equal to 1.

In [ ]:
Depolarizing(0.5, [0, 1, 2], dimension=2)
Depolarizing(0.5, [0, 1, 2], gates=[H, Rx, U])
Depolarizing(0.5, [0, 1, 2], dimension=2, gates=[CNOT, CZ])

### Adding noise to the circuit

In [ ]:
# using direct instantiation
circ = QCircuit([H(0), CNOT(0,1), Y(1), BasisMeasure([0,1], shots=100), Depolarizing(0.3, [0], gates=[H])])

# using add
circ = QCircuit([H(0), CNOT(0,1), Y(1), BasisMeasure([0,1], shots=100)])
circ.add([Depolarizing(0.3, [0]), Depolarizing(0.13, [1])])

# circuit.noises
print(circuit.noises)

# circuit.pretty_print()
circuit.pretty_print()

# circuit.without_noises()
print(circuit.without_noises)

# circuit.to_other_language(Language.BRAKET)
noisy_braket_circuit = circuit.to_other_language(Language.BRAKET)
print(noisy_braket_circuit)


In [95]:
# build your circuit
circuit = QCircuit(3, label="Noise-Simulation")
circuit.add([H(0), CNOT(0,1), T(2)])

In [96]:
# define a depolarizing noise channel
depolarizing_noise = Depolarizing(prob=0.4, targets=[0, 1])

In [97]:
# apply depolarizing noise to the circuit
circuit.add(depolarizing_noise)

In [98]:
# add measurements
circuit.add(BasisMeasure([0, 1, 2], shots=1024))

In [99]:
circuit.pretty_print()

QCircuit Noise-Simulation: Size (Qubits,Cbits) = (3, 3), Nb instructions = 4
Depolarizing noise: probability 0.4 on qubits [0, 1]
     ┌───┐     ┌─┐   
q_0: ┤ H ├──■──┤M├───
     └───┘┌─┴─┐└╥┘┌─┐
q_1: ─────┤ X ├─╫─┤M├
     ┌───┐└┬─┬┘ ║ └╥┘
q_2: ┤ T ├─┤M├──╫──╫─
     └───┘ └╥┘  ║  ║ 
c: 3/═══════╩═══╩══╩═
            2   0  1 


### Running on noisy simulators

In [ ]:
# device.is_noisy()

# which device is supported

# run as any other computation

# observable and sample jobs only

In [100]:
# select the local noise simulator
device = AWSDevice.BRAKET_LOCAL_SIMULATOR

In [101]:
# run the circuit on the local simulator
result = run(circuit, device)

In [8]:
print(result)

Result: AWSDevice, BRAKET_LOCAL_SIMULATOR
Counts: [478, 0, 0, 0, 0, 0, 546, 0]
Probabilities: [0.46679688 0.         0.         0.         0.         0.  0.53320312 0.        ]
State: 110, Index: 6, Count: 546, Probability: 0.533203125
State: 000, Index: 0, Count: 478, Probability: 0.466796875
Error: None


